## 작업형2 기출 유형(심화)
- 본 문제는 변형한 심화 문제 입니다.
- 오리지널 3회 기출 유형을 보고 싶은 분은 아래 클래스-커리큘럼 탭에 무료공개(3회 작업형2)로 영상과 데이터셋을 올려놨어요!
- https://class101.net/products/467P0ZPH0lVX9FwFBDz7

**여행 보험 패키지 상품을 구매할 확률 값을 구하시오**
- 예측할 값(y): TravelInsurance (여행보험 패지지를 구매 했는지 여부 0:구매안함, 1:구매)
- 평가: roc-auc 평가지표
- data: t2-1-train.csv, t2-1-test.csv

제출 형식

```
id,TravelInsurance
0,0.3
1,0.48
2,0.3
3,0.83
```

## Baseline
**3회 기출문제에서 데이터 셋을 편집해 조금 더 어렵게 만들었어요**
- 결측치 추가
- Employment Type 컬럼에 카테고리 추가
- sample_submission 파일은 제공된 적 없음(3회 때 제출 형식에 대한 이슈가 있어 제공하거나 제출 형식을 명확하게 설명할 가능성 있어 보임)

In [1]:
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score

train = pd.read_csv('../dataset/t2-1-train.csv')
test = pd.read_csv('../dataset/t2-1-test.csv')
train.shape, test.shape

((1490, 10), (497, 9))

---
## My Code

---
## Solution

In [2]:
train.head()

,id,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,10000,28,Private Sector/Self Employed,Yes,1250000.0,6,1,No,No,0
1,10001,31,Private Sector/Self Employed,Yes,1250000.0,7,1,No,No,0
2,10002,29,Private Sector/Self Employed,Yes,1200000.0,7,0,No,No,1
3,10003,33,Government Sector,Yes,650000.0,6,1,No,No,1
4,10004,28,Private Sector/Self Employed,Yes,800000.0,6,0,No,Yes,1


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   1490 non-null   int64  
 1   Age                  1490 non-null   int64  
 2   Employment Type      1490 non-null   object 
 3   GraduateOrNot        1490 non-null   object 
 4   AnnualIncome         1486 non-null   float64
 5   FamilyMembers        1490 non-null   int64  
 6   ChronicDiseases      1490 non-null   int64  
 7   FrequentFlyer        1490 non-null   object 
 8   EverTravelledAbroad  1490 non-null   object 
 9   TravelInsurance      1490 non-null   int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 116.5+ KB


실제 시험에서는 train과 test 카테고리가 같았어요. 만약 test데이터에 새로운 카테고리가 있다면 어떻게 풀어야 할까요?

In [4]:
# 카테고리 수 확인
train.describe(include="object")

,Employment Type,GraduateOrNot,FrequentFlyer,EverTravelledAbroad
count,1490,1490,1490,1490
unique,2,2,2,2
top,Private Sector/Self Employed,Yes,No,No
freq,1056,1270,1175,1209


In [5]:
# 카테고리 수 확인
test.describe(include="object")

,Employment Type,GraduateOrNot,FrequentFlyer,EverTravelledAbroad
count,497,497,497,497
unique,3,2,2,2
top,Private Sector/Self Employed,Yes,No,No
freq,360,422,395,398


In [6]:
train['Employment Type'].value_counts()

Private Sector/Self Employed    1056
Government Sector                434
Name: Employment Type, dtype: int64

In [7]:
test['Employment Type'].value_counts()

Private Sector/Self Employed    360
Government Sector               134
Casual employment                 3
Name: Employment Type, dtype: int64

In [8]:
# 수치형 통계 값
train.describe(exclude="object")

,id,Age,AnnualIncome,FamilyMembers,ChronicDiseases,TravelInsurance
count,1490.000000,1490.000000,1.486000e+03,1490.000000,1490.000000,1490.000000
mean,10744.500000,29.600000,9.311238e+05,4.755705,0.280537,0.352349
std,430.270264,2.887829,3.764874e+05,1.603613,0.449412,0.477862
min,10000.000000,25.000000,3.000000e+05,2.000000,0.000000,0.000000
25%,10372.250000,28.000000,6.000000e+05,4.000000,0.000000,0.000000
50%,10744.500000,29.000000,9.000000e+05,5.000000,0.000000,0.000000
75%,11116.750000,32.000000,1.250000e+06,6.000000,1.000000,1.000000
max,11489.000000,35.000000,1.800000e+06,9.000000,1.000000,1.000000


In [9]:
# 수치형 통계 값
test.describe(exclude="object")

,id,Age,AnnualIncome,FamilyMembers,ChronicDiseases
count,497.000000,497.000000,4.940000e+02,497.000000,497.000000
mean,248.000000,29.800805,9.393725e+05,4.744467,0.269618
std,143.615807,2.986286,3.792988e+05,1.629211,0.444208
min,0.000000,25.000000,3.000000e+05,2.000000,0.000000
25%,124.000000,28.000000,6.000000e+05,4.000000,0.000000
50%,248.000000,29.000000,9.000000e+05,5.000000,0.000000
75%,372.000000,33.000000,1.250000e+06,6.000000,1.000000
max,496.000000,35.000000,1.750000e+06,9.000000,1.000000


실제 시험에서는 결측치가 없었어요. 만약 결측치가 있다면 어떻게 풀어야 할까요?

In [10]:
# 결측치 확인
train.isnull().sum()

id                     0
Age                    0
Employment Type        0
GraduateOrNot          0
AnnualIncome           4
FamilyMembers          0
ChronicDiseases        0
FrequentFlyer          0
EverTravelledAbroad    0
TravelInsurance        0
dtype: int64

In [11]:
# 결측치 확인
test.isnull().sum()

id                     0
Age                    0
Employment Type        0
GraduateOrNot          0
AnnualIncome           3
FamilyMembers          0
ChronicDiseases        0
FrequentFlyer          0
EverTravelledAbroad    0
dtype: int64

In [12]:
# target
train['TravelInsurance'].value_counts()

0    965
1    525
Name: TravelInsurance, dtype: int64

In [13]:
pd.unique(train['TravelInsurance'])

array([0, 1])

## Data pre-processing

In [14]:
# 결측치 처리
train['AnnualIncome'] = train['AnnualIncome'].fillna(train['AnnualIncome'].mean())
test['AnnualIncome'] = test['AnnualIncome'].fillna(test['AnnualIncome'].mean())

In [15]:
# target값 변수에 옮기기
target = train.pop('TravelInsurance')

In [16]:
# 데이터 합치기
df = pd.concat([train, test])
df.shape

(1987, 9)

In [17]:
# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder

cols = df.select_dtypes(include='object').columns
le = LabelEncoder()

for col in cols:
    df[col] = le.fit_transform(df[col])
    
df.head()

,id,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad
0,10000,28,2,1,1250000.0,6,1,0,0
1,10001,31,2,1,1250000.0,7,1,0,0
2,10002,29,2,1,1200000.0,7,0,0,0
3,10003,33,1,1,650000.0,6,1,0,0
4,10004,28,2,1,800000.0,6,0,0,1


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1987 entries, 0 to 496
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   1987 non-null   int64  
 1   Age                  1987 non-null   int64  
 2   Employment Type      1987 non-null   int64  
 3   GraduateOrNot        1987 non-null   int64  
 4   AnnualIncome         1987 non-null   float64
 5   FamilyMembers        1987 non-null   int64  
 6   ChronicDiseases      1987 non-null   int64  
 7   FrequentFlyer        1987 non-null   int64  
 8   EverTravelledAbroad  1987 non-null   int64  
dtypes: float64(1), int64(8)
memory usage: 155.2 KB


In [19]:
df.shape, train.shape, test.shape, train.shape[0], test.shape[0]

((1987, 9), (1490, 9), (497, 9), 1490, 497)

In [20]:
# train test 다시 분리
train = df[:train.shape[0]].copy()
test = df[train.shape[0]:].copy()
train.shape, test.shape

((1490, 9), (497, 9))

In [21]:
# 스케일
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

train['AnnualIncome'] = scaler.fit_transform(train[['AnnualIncome']])
test['AnnualIncome'] = scaler.transform(test[['AnnualIncome']])

## 검증 데이터 분리

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(train, target, test_size=0.2, random_state=2022)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((1192, 9), (298, 9), (1192,), (298,))

## 모델 학습 및 평가

##### 의사결정나무

In [23]:
# 의사결정나무
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train, Y_train)
pred = model.predict_proba(X_test)
pred.shape

(298, 2)

In [24]:
# 평가
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_test, pred[:, 1])

0.6887878787878788

##### 랜덤포레스트

In [25]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, Y_train)
pred = model.predict_proba(X_test)
pred.shape

(298, 2)

In [26]:
roc_auc_score(Y_test, pred[:,1])

0.752449494949495

##### xgboost

In [27]:
import xgboost as xgb

model = xgb.XGBRFClassifier(random_state=2022, use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, Y_train)
pred = model.predict_proba(X_test)
pred.shape

(298, 2)

In [28]:
roc_auc_score(Y_test, pred[:,1])

0.7652777777777777

## 예측

In [29]:
# test 데이터 예측
model = RandomForestClassifier(random_state=2022)
model.fit(X_train, Y_train)
pred = model.predict_proba(test)

In [30]:
# 예측한 데이터 -> 데이터프레임으로
submit = pd.DataFrame()
submit['id'] = test['id']
submit['TravelInsurance'] = pred[:,1]
submit.head()

,id,TravelInsurance
0,0,0.30
1,1,0.48
2,2,0.30
3,3,0.83
4,4,0.28


In [31]:
# csv 저장
submit.to_csv('2022.csv', index=False)

In [32]:
# csv 확인
pd.read_csv('2022.csv')

,id,TravelInsurance
0,0,0.30
1,1,0.48
2,2,0.30
3,3,0.83
4,4,0.28
...,...,...
492,492,0.13
493,493,0.96
494,494,0.16
495,495,0.71


## 추가
- 만약 sample_submission이 주어진다면

In [33]:
pred.shape, pred[:,1].shape

((497, 2), (497,))

In [34]:
sample_submission = pd.read_csv('../dataset/t2-1-sample_submission.csv')
sample_submission['TravelInsurance'] = pred[:,1]
sample_submission.to_csv('2022.csv', index=False)